In [ ]:
import sys
print(sys.executable)


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import vegafusion as vf
vf.enable()
import json

In [ ]:
data = pd.read_csv('../Data/AirQualityHealthRisks.csv')
EuropeData = gpd.read_file('../Data/europe.geojson')

country_mapping = {
    "Türkiye": "Turkey",
    "North Macedonia": "The former Yugoslav Republic of Macedonia",
    "Czechia": "Czech Republic",
    "United Kingdom": "United Kingdom"
}
data["Country Or Territory"] = data["Country Or Territory"].replace(country_mapping)

deduplicated_data = data.drop_duplicates(subset=["Country Or Territory", "Year", "Air Pollutant"])
aggregated_data = deduplicated_data.groupby(
    ["Country Or Territory", "Year", "Air Pollutant"], as_index=False
).agg({
    "Premature Deaths": "sum",
    "Years Of Life Lost": "sum",
    "Population": "mean"  
})

merged_data = EuropeData.merge(
    aggregated_data,
    left_on="NAME",
    right_on="Country Or Territory",
    how="left"
)

# Add a column for lives lost per person
merged_data["Lives Lost per Person"] = merged_data["Years Of Life Lost"] / merged_data["Population"]

# All Countries, EU, EEA
specific_categories = ["All Countries", "European Union Countries", "European Environment Agency Member Countries"]
category_data = aggregated_data[aggregated_data["Country Or Territory"].isin(specific_categories)]
category_totals = category_data.groupby("Country Or Territory").agg({
    "Premature Deaths": "sum",
    "Years Of Life Lost": "sum",
    "Population": "mean"
}).reset_index()

# merged_data.to_csv("Cleaned_Merged_Data.csv", index=False)
# category_totals.to_csv("Category_Totals.csv", index=False)



In [ ]:

merged_data["Lives Lost per Person"] = merged_data["Years Of Life Lost"] / merged_data["Population"]

choropleth = alt.Chart(merged_data).mark_geoshape(
    stroke='white', strokeWidth=0.5
).encode(
    color=alt.Color(
        "Premature Deaths:Q",
        scale=alt.Scale(scheme="reds"),
        title="Premature Deaths",
        legend=alt.Legend(title="Premature Deaths", orient="bottom-right")
    ),
    tooltip=[
        alt.Tooltip("NAME:N", title="Country"),
        alt.Tooltip("Premature Deaths:Q", title="Premature Deaths", format=",.0f"),
        alt.Tooltip("Years Of Life Lost:Q", title="Years of Life Lost", format=",.0f"),
        alt.Tooltip("Lives Lost per Person:Q", title="Lives Lost per Person", format=".6f")
    ]
).project(
    type="mercator",
    scale=600,
    center=[10, 50]
).properties(
    width=1000,
    height=700,
    title=alt.TitleParams(
        "Premature Deaths and Lives Lost per Person in Europe",
        fontSize=20,
        fontWeight="bold",
        anchor="middle"
    )
).configure_view(
    stroke=None
).configure_legend(
    titleFontSize=14,
    labelFontSize=12,
    gradientLength=200
)

choropleth.show()
